In [1]:
import spacy
from spacy.tokens import Token
from spacy import displacy
import pandas as pd
import pickle
import random

In [2]:
Token.set_extension('ref_n', default='', force = True)
Token.set_extension('ref_t', default='', force = True)

#nlp = spacy.load("../NER/models/ner_v1.0")
#nlp = spacy.load('en_core_web_lg')

In [3]:
file = open("../stored_data/docs.obj",'rb')
docs = pickle.load(file)
file.close()

In [4]:
from spacy.matcher import Matcher

nlp = spacy.load('en_core_web_sm')
matcher = Matcher(nlp.vocab)

C:\Users\aubin\anaconda3\envs\nlp-new\lib\site-packages\spacy\util.py:833: UserWarning: [W095] Model 'en_core_web_sm' (3.1.0) was trained with spaCy v3.1 and may not be 100% compatible with the current version (3.2.1). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [11]:
entities = nlp.get_pipe('ner').labels

In [6]:
pattern = [{'_': {'ref_t': {'IN': entities}}}, # subject
        {'OP': '*'},
        {'_': {'ref_t': {'IN': entities}}}] # object
matcher.add(f"any_verb", [pattern])

In [7]:
text = "Fairchild Corp acquired Fujitsu."

doc = nlp(text)
matches = matcher(doc)
print(matches)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

[]


In [8]:
id = random.sample(list(docs.keys()), 1)
random_doc = docs[id[0]]
full_doc = ""
for sent in random_doc:
    full_doc += sent

In [9]:
for sentence in random_doc:
    doc = nlp(sentence)
    matches = matcher(doc)
    print(matches)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # Get string representation
        span = doc[start:end]  # The matched span
        print(match_id, string_id, start, end, span.text)

[]
[]
[]
[]


In [17]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_lg")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern = [
    {"LOWER": "hello"}, 
    {"IS_PUNCT": True}, 
    {"LOWER": "world"}]
matcher.add("HelloWorld", [pattern])

doc = nlp("Hello, world! Hello world!")
matches = matcher(doc)
for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

15578876784678163569 HelloWorld 0 3 Hello, world


In [18]:
from spacy.matcher import Matcher

matcher = Matcher(nlp.vocab)

acq_synonyms = ['acquire', 'buy', 'purchase']
pattern = [{'_': {'ref_t': 'ORG'}}, # subject
           {'_': {'ref_t': {'NOT_IN': ['ORG']}}, 'OP': '*'},
           {'POS': 'VERB', 'LEMMA': {'IN': acq_synonyms}},
           {'_': {'ref_t': {'NOT_IN': ['ORG']}}, 'OP': '*'},
           {'_': {'ref_t': 'ORG'}}] # object
matcher.add('acquires', [pattern])

In [19]:
def extract_rel_match(doc, matcher):
    for sent in doc.sents:
        for match_id, start, end in matcher(sent):
            span = sent[start:end]  # matched span
            pred = nlp.vocab.strings[match_id] # rule name
            subj, obj = span[0], span[-1]
            if pred.startswith('rev-'): # reversed relation
                subj, obj = obj, subj
                pred = pred[4:]
            yield ((subj._.ref_n, subj._.ref_t), pred, 
                   (obj._.ref_n, obj._.ref_t))

In [20]:
text = "Fairchild Corp was acquired by Fujitsu."
print(*extract_rel_match(nlp(text), matcher), sep='\n')